In [8]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [9]:
def true_reward(theta, theta_dt, torque):
    return -(theta**2 + 0.1 * theta_dt**2 + 0.001 * torque**2)

In [10]:
class PendulumRewardDataset(Dataset):
    def __init__(self, size=10000):
        self.size = size
        self.data = []

        for _ in range(size):
            theta = np.random.uniform(-np.pi, np.pi)
            theta_dt = np.random.uniform(-8.0, 8.0)
            torque = np.random.uniform(-2.0, 2.0)
            reward = true_reward(theta, theta_dt, torque)
            self.data.append((theta, theta_dt, torque, reward))
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        theta, theta_dt, torque, reward = self.data[idx]
        x = torch.tensor([theta, theta_dt, torque], dtype=torch.float32)
        y = torch.tensor([reward], dtype=torch.float32)
        return x, y

In [11]:
dataset = PendulumRewardDataset(size=10000)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [12]:
class RewardModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def forward(self, x):
        return self.net(x)

In [13]:
def train_reward_model(model, dataloader, epochs=10, lr=1e-3, device='cpu'):
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            pred = model(x)
            loss = criterion(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(dataloader)
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

In [14]:
model = RewardModel()
train_reward_model(model, dataloader, epochs=10, lr=1e-3)

Epoch 1, Loss: 10.9741
Epoch 2, Loss: 1.9405
Epoch 3, Loss: 0.3938
Epoch 4, Loss: 0.1880
Epoch 5, Loss: 0.1291
Epoch 6, Loss: 0.0797
Epoch 7, Loss: 0.0514
Epoch 8, Loss: 0.0363
Epoch 9, Loss: 0.0268
Epoch 10, Loss: 0.0212


In [17]:
def test_model_prediction(model, theta, theta_dt, torque):
    model.eval()
    
    # Input tensor
    x = torch.tensor([[theta, theta_dt, torque]], dtype=torch.float32)
    
    # Model prediction
    with torch.no_grad():
        predicted_reward = model(x).item()
    
    # Ground truth
    actual_reward = true_reward(theta, theta_dt, torque)
    
    print(f"Input: theta={theta:.3f}, theta_dt={theta_dt:.3f}, torque={torque:.3f}")
    print(f"Predicted Reward: {predicted_reward:.4f}")
    print(f"True Reward:      {actual_reward:.4f}")
    print(f"Error:            {abs(predicted_reward - actual_reward):.4f}")

In [18]:
test_model_prediction(model, theta=1.0, theta_dt=0.5, torque=-1.5)

Input: theta=1.000, theta_dt=0.500, torque=-1.500
Predicted Reward: -0.8990
True Reward:      -1.0272
Error:            0.1283
